In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

from tqdm import tqdm

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [ ]:
def set_seed(seed: int):
  torch.manual_seed(seed)
  torch.cuda.manual_seed(seed)
  np.random.seed(seed)

  return seed
seed = set_seed(810)

In [ ]:
# 데이터 로드
final_dfs = pd.read_csv('/content/drive/MyDrive/1데이콘/Track1알고리즘부문:K리그-서울시립대공개AI경진대회/dataset/final_dfs.csv')

In [ ]:
final_df = final_dfs[~final_dfs['game_episode'].str.contains('_aug')]

# Dataset & DataLoader

In [ ]:
feature_cols = final_df.columns.drop(['game_episode', 'target_dx', 'target_dy'])
target_cols = ['target_dx', 'target_dy']

In [ ]:
# train test 분할
train_df = final_df[final_df['is_train'] == 1]
test_df = final_df[final_df['is_train'] == 0]

In [ ]:
print(train_df.shape, test_df.shape)

(15435, 111) (2414, 111)


In [ ]:
class EpisodeDataset(Dataset):
    def __init__(self, df, feature_cols, target_cols):
        self.feature_cols = feature_cols
        self.target_cols = target_cols

        self.episodes = []
        self.targets = []

        for ep, g in df.groupby("game_episode"):
            g = g.sort_values("time_seconds").reset_index(drop=True)

            seq = torch.tensor(g[feature_cols].values, dtype=torch.float32)
            tgt = torch.tensor(g[target_cols].iloc[-1].values,dtype=torch.float32)

            self.episodes.append(seq)
            self.targets.append(tgt)

    def __len__(self):
        return len(self.episodes)

    def __getitem__(self, idx):
        seq = self.episodes[idx]     # [T, 110]
        tgt = self.targets[idx]      # [2]
        length = seq.size(0)
        return seq, length, tgt


def collate_fn(batch):
    seqs, lengths, tgts = zip(*batch)
    lengths = torch.tensor(lengths, dtype=torch.long)
    padded = pad_sequence(seqs, batch_first=True)   # [B, Tmax, F]
    tgts = torch.stack(tgts)                 # [B, 2]

    return padded, lengths, tgts

In [ ]:
episodes_unique = final_df["game_episode"].unique()

train_eps, val_eps = train_test_split(episodes_unique, test_size=0.2, random_state=seed)

train_df = final_df[final_df["game_episode"].isin(train_eps)].reset_index(drop=True)
val_df = final_df[final_df["game_episode"].isin(val_eps)].reset_index(drop=True)

In [ ]:
# target_mode = 'delta'
batch_size = 32

In [ ]:
# 데이터셋 & 데이터로더
train_dataset = EpisodeDataset(train_df, feature_cols, target_cols)
val_dataset = EpisodeDataset(val_df, feature_cols, target_cols)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn, drop_last=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

# LSTM Modeling

In [ ]:
# batch_size = 32
EPOCHS = 100
LR = 1e-3
input_dim = 108
hidden_dim = 64

In [ ]:
class LSTMBaseline(nn.Module):
    def __init__(self, input_dim=input_dim, hidden_dim=hidden_dim):
        super().__init__()
        self.lstm = nn.LSTM(
            input_size=input_dim,
            hidden_size=hidden_dim,
            num_layers=1,
            batch_first=True,
        )
        self.fc = nn.Linear(hidden_dim, 2)  # (x_norm, y_norm)

    def forward(self, x, lengths):
        # x: [B, T, 2], lengths: [B]
        packed = pack_padded_sequence(
            x, lengths.cpu(), batch_first=True, enforce_sorted=False
        )
        _, (h_n, _) = self.lstm(packed)
        h_last = h_n[-1]      # [B, H] 마지막 layer의 hidden state
        out = self.fc(h_last) # [B, 2]
        return out

model = LSTMBaseline(input_dim=input_dim, hidden_dim=hidden_dim).to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

In [ ]:
best_dist = float("inf")
best_model_state = None

for epoch in range(1, EPOCHS + 1):
    # --- Train ---
    model.train()
    total_loss = 0.0

    for seq, length, tgt in tqdm(train_loader):
        seq, length, tgt = seq.to(device), length.to(device), tgt.to(device)

        optimizer.zero_grad()
        pred = model(seq, length)
        loss = criterion(pred, tgt)
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * seq.size(0)

    train_loss = total_loss / len(train_loader.dataset)

    # --- Valid: 평균 유클리드 거리 ---
    # --- Valid: dx, dy 벡터 거리 ---
    model.eval()
    dists = []

    with torch.no_grad():
      for seq, length, tgt in tqdm(val_loader):
          seq, length, tgt = seq.to(device), length.to(device), tgt.to(device)
          pred = model(seq, length)

          diff = pred - tgt
          dist = torch.sqrt(torch.sum(diff * diff, dim=1))  # [B]
          dists.append(dist.cpu().numpy())

    mean_dist = np.concatenate(dists).mean()


    print(
        f"[Epoch {epoch}] "
        f"train_loss={train_loss:.4f} | "
        f"valid_mean_dist={mean_dist:.4f}"
    )

    # ----- BEST MODEL 업데이트 -----
    if mean_dist < best_dist:
        best_dist = mean_dist
        best_model_state = model.state_dict().copy()
        print(f" --> Best model updated! (dist={best_dist:.4f})")

100%|██████████| 112/112 [00:00<00:00, 369.71it/s]


[Epoch 1] train_loss=1.0096 | valid_mean_dist=1.0951
 --> Best model updated! (dist=1.0951)


100%|██████████| 112/112 [00:00<00:00, 932.58it/s]


[Epoch 2] train_loss=1.0052 | valid_mean_dist=1.1013


100%|██████████| 112/112 [00:00<00:00, 949.83it/s]


...

[Epoch 98] train_loss=1.0033 | valid_mean_dist=1.0840


100%|██████████| 112/112 [00:00<00:00, 865.52it/s]


[Epoch 99] train_loss=1.0036 | valid_mean_dist=1.0933


100%|██████████| 112/112 [00:00<00:00, 922.88it/s]

[Epoch 100] train_loss=1.0035 | valid_mean_dist=1.0863


# 추론

In [ ]:
class EpisodeDataset(Dataset):
    def __init__(self, df, feature_cols, target_cols):
        self.feature_cols = feature_cols
        self.target_cols = target_cols

        self.episodes = []
        for ep, g in df.groupby("game_episode"):

            seq = g[feature_cols].values.astype(np.float32)

            # test에는 실제 target 없을 수 있어서 처리
            if set(target_cols).issubset(g.columns):
                tgt = g[target_cols].values[-1].astype(np.float32)
            else:
                tgt = np.zeros(len(target_cols), dtype=np.float32)

            self.episodes.append((seq, tgt))

    def __len__(self):
        return len(self.episodes)

    def __getitem__(self, idx):
        seq, tgt = self.episodes[idx]
        length = len(seq)
        return torch.tensor(seq), length, torch.tensor(tgt)

In [ ]:
test_dataset = EpisodeDataset(
    test_df,
    feature_cols=feature_cols,
    target_cols=["target_dx", "target_dy"]  # 없어도 됨
)

test_loader = DataLoader(
    test_dataset,
    batch_size=64,
    shuffle=False,
    collate_fn=collate_fn
)

In [ ]:
model.eval()
preds_x = []
preds_y = []

with torch.no_grad():
    for x, length, y in tqdm(test_loader, desc="Inference"):
        x = x.to(device)              # [B, T, 2]
        length = length.to(device) # [B]

        pred = model(x, length)     # [B, 2]

        # 정규화된 출력 → 원래 스케일 복원
        pred = pred.cpu().numpy()    # shape: (B, 2)

        # 스케일링(105, 68) 복구 후 리스트에 저장
        px = pred[:, 0] * 105.0
        py = pred[:, 1] * 68.0

        preds_x.extend(px.tolist())
        preds_y.extend(py.tolist())

print("Inference Done.")

Inference: 100%|██████████| 38/38 [00:00<00:00, 370.27it/s]

Inference Done.


# submission 생성

In [ ]:
submission = pd.read_csv(
    "/content/drive/MyDrive/1데이콘/Track1알고리즘부문:K리그-서울시립대공개AI경진대회/dataset/sample_submission.csv")

submission["end_x"] = preds_x
submission["end_y"] = preds_y

submission.to_csv("/content/drive/MyDrive/1데이콘/Track1알고리즘부문:K리그-서울시립대공개AI경진대회/submission4.csv", index=False)
print("Saved: baseline_submit.csv")

Saved: baseline_submit.csv


# baseline

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tqdm import tqdm

import torch
from torch import nn
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence
from torch.utils.data import Dataset, DataLoader
import os
TRAIN_PATH = "/content/drive/MyDrive/1데이콘/Track1알고리즘부문:K리그-서울시립대공개AI경진대회/dataset/train.csv"
BATCH_SIZE = 64
EPOCHS = 5
LR = 1e-3
HIDDEN_DIM = 64
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", DEVICE)

df = pd.read_csv(TRAIN_PATH)
df = df.sort_values(["game_episode", "time_seconds"]).reset_index(drop=True)

episodes = []
targets = []

for _, g in tqdm(df.groupby("game_episode")):
    g = g.reset_index(drop=True)
    if len(g) < 2:
        continue

    # 정규화된 좌표 준비
    sx = g["start_x"].values / 105.0
    sy = g["start_y"].values / 68.0
    ex = g["end_x"].values   / 105.0
    ey = g["end_y"].values   / 68.0

    coords = []
    for i in range(len(g)):
        # 항상 start는 들어감
        coords.append([sx[i], sy[i]])
        # 마지막 행 이전까지만 end를 넣음 (마지막 end는 타깃이므로)
        if i < len(g) - 1:
            coords.append([ex[i], ey[i]])

    seq = np.array(coords, dtype="float32")        # [T, 2]
    target = np.array([ex[-1], ey[-1]], dtype="float32")  # 마지막 행 end_x, end_y

    episodes.append(seq)
    targets.append(target)

print("에피소드 수 : ", len(episodes))

class EpisodeDataset(Dataset):
    def __init__(self, episodes, targets):
        self.episodes = episodes
        self.targets = targets

    def __len__(self):
        return len(self.episodes)

    def __getitem__(self, idx):
        seq = torch.tensor(self.episodes[idx])   # [T, 2]
        tgt = torch.tensor(self.targets[idx])    # [2]
        length = seq.size(0)
        return seq, length, tgt

def collate_fn(batch):
    seqs, lengths, tgts = zip(*batch)
    lengths = torch.tensor(lengths, dtype=torch.long)
    padded = pad_sequence(seqs, batch_first=True)  # [B, T, 2]
    tgts = torch.stack(tgts, dim=0)                # [B, 2]
    return padded, lengths, tgts

# 에피소드 단위 train / valid split
idx_train, idx_valid = train_test_split(
    np.arange(len(episodes)), test_size=0.2, random_state=42
)

episodes_train = [episodes[i] for i in idx_train]
targets_train  = [targets[i]  for i in idx_train]
episodes_valid = [episodes[i] for i in idx_valid]
targets_valid  = [targets[i]  for i in idx_valid]

train_loader = DataLoader(
    EpisodeDataset(episodes_train, targets_train),
    batch_size=BATCH_SIZE,
    shuffle=True,
    collate_fn=collate_fn,
)

valid_loader = DataLoader(
    EpisodeDataset(episodes_valid, targets_valid),
    batch_size=BATCH_SIZE,
    shuffle=False,
    collate_fn=collate_fn,
)

print("train episodes:", len(episodes_train), "valid episodes:", len(episodes_valid))

class LSTMBaseline(nn.Module):
    def __init__(self, input_dim=2, hidden_dim=64):
        super().__init__()
        self.lstm = nn.LSTM(
            input_size=input_dim,
            hidden_size=hidden_dim,
            num_layers=1,
            batch_first=True,
        )
        self.fc = nn.Linear(hidden_dim, 2)  # (x_norm, y_norm)

    def forward(self, x, lengths):
        # x: [B, T, 2], lengths: [B]
        packed = pack_padded_sequence(
            x, lengths.cpu(), batch_first=True, enforce_sorted=False
        )
        _, (h_n, _) = self.lstm(packed)
        h_last = h_n[-1]      # [B, H] 마지막 layer의 hidden state
        out = self.fc(h_last) # [B, 2]
        return out

model = LSTMBaseline(input_dim=2, hidden_dim=HIDDEN_DIM).to(DEVICE)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

best_dist = float("inf")
best_model_state = None

for epoch in range(1, EPOCHS + 1):
    # --- Train ---
    model.train()
    total_loss = 0.0

    for X, lengths, y in tqdm(train_loader):
        X, lengths, y = X.to(DEVICE), lengths.to(DEVICE), y.to(DEVICE)

        optimizer.zero_grad()
        pred = model(X, lengths)
        loss = criterion(pred, y)
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * X.size(0)

    train_loss = total_loss / len(train_loader.dataset)

    # --- Valid: 평균 유클리드 거리 ---
    model.eval()
    dists = []

    with torch.no_grad():
        for X, lengths, y in tqdm(valid_loader):
            X, lengths, y = X.to(DEVICE), lengths.to(DEVICE), y.to(DEVICE)
            pred = model(X, lengths)

            pred_np = pred.cpu().numpy()
            true_np = y.cpu().numpy()

            pred_x = pred_np[:, 0] * 105.0
            pred_y = pred_np[:, 1] * 68.0
            true_x = true_np[:, 0] * 105.0
            true_y = true_np[:, 1] * 68.0

            dist = np.sqrt((pred_x - true_x) ** 2 + (pred_y - true_y) ** 2)
            dists.append(dist)

    mean_dist = np.concatenate(dists).mean()  # 평균 유클리드 거리

    print(
        f"[Epoch {epoch}] "
        f"train_loss={train_loss:.4f} | "
        f"valid_mean_dist={mean_dist:.4f}"
    )

    # ----- BEST MODEL 업데이트 -----
    if mean_dist < best_dist:
        best_dist = mean_dist
        best_model_state = model.state_dict().copy()
        print(f" --> Best model updated! (dist={best_dist:.4f})")

# Best Model Load
model.load_state_dict(best_model_state)
model.eval()

test_meta = pd.read_csv("/content/drive/MyDrive/1데이콘/Track1알고리즘부문:K리그-서울시립대공개AI경진대회/dataset/test.csv")
submission = pd.read_csv("/content/drive/MyDrive/1데이콘/Track1알고리즘부문:K리그-서울시립대공개AI경진대회/dataset/sample_submission.csv")

submission = submission.merge(test_meta, on="game_episode", how="left")

preds_x, preds_y = [], []

path = "/content/drive/MyDrive/1데이콘/Track1알고리즘부문:K리그-서울시립대공개AI경진대회/dataset"
for _, row in tqdm(submission.iterrows(), total=len(submission)):
    # g = pd.read_csv(path + row["path"]).reset_index(drop=True)
    # re_path = os.path.normpath(path + row['path'])
    p = row["path"]
    print(p)
    main = p.rsplit('.', 1)[0]     # 확장자 제거한 부분
    ext  = p.rsplit('.', 1)[1]     # 확장자
    print(main)
    print(ext)
    main = main.replace('.', '/')
    file_path = main + '.' + ext

    g = pd.read_csv(path + file_path).reset_index(drop=True)
    # 정규화된 좌표 준비
    sx = g["start_x"].values / 105.0
    sy = g["start_y"].values / 68.0
    ex = g["end_x"].values / 105.0
    ey = g["end_y"].values / 68.0

    coords = []
    for i in range(len(g)):
        # start는 항상 존재하므로 그대로 사용
        coords.append([sx[i], sy[i]])
        # 마지막 행은 end_x가 NaN이므로 자동으로 제외됨
        if i < len(g) - 1:
            coords.append([ex[i], ey[i]])

    seq = np.array(coords, dtype="float32")  # [T, 2]

    x = torch.tensor(seq).unsqueeze(0).to(DEVICE)      # [1, T, 2]
    length = torch.tensor([seq.shape[0]]).to(DEVICE)   # [1]

    with torch.no_grad():
        pred = model(x, length).cpu().numpy()[0]       # [2], 정규화 좌표

    preds_x.append(pred[0] * 105.0)
    preds_y.append(pred[1] * 68.0)
print("Inference Done.")


submission["end_x"] = preds_x
submission["end_y"] = preds_y
submission[["game_episode", "end_x", "end_y"]].to_csv("/content/drive/MyDrive/1데이콘/Track1알고리즘부문:K리그-서울시립대공개AI경진대회/submission.csv", index=False)
print("Saved: baseline_submit.csv")

Using device: cuda


100%|██████████| 15435/15435 [00:04<00:00, 3595.63it/s]


에피소드 수 :  15428
train episodes: 12342 valid episodes: 3086


100%|██████████| 49/49 [00:00<00:00, 206.30it/s]


[Epoch 1] train_loss=0.0941 | valid_mean_dist=20.5886
 --> Best model updated! (dist=20.5886)


...

[Epoch 4] train_loss=0.0340 | valid_mean_dist=17.9962
 --> Best model updated! (dist=17.9962)


100%|██████████| 49/49 [00:00<00:00, 221.43it/s]


[Epoch 5] train_loss=0.0333 | valid_mean_dist=18.0309


100%|██████████| 2414/2414 [23:46<00:00,  1.69it/s]

Inference Done.
Saved: baseline_submit.csv


In [ ]:
preds_x

[np.float32(71.9549),
 np.float32(44.242043),
 np.float32(39.04921),
 np.float32(58.790363),
 np.float32(79.88367),
 np.float32(76.65093),
 np.float32(59.093723),
 np.float32(72.221855),
 np.float32(48.981632),
 np.float32(67.01466),
 np.float32(58.37162),
 np.float32(82.11038),
 np.float32(88.25367),
 np.float32(44.96841),
 np.float32(61.955944),
 np.float32(69.98612),
 np.float32(50.483566),
 np.float32(60.079887),
 np.float32(39.550938),
 np.float32(68.53436),
 np.float32(58.269485),
 np.float32(76.74692),
 np.float32(96.716606),
 np.float32(102.62544),
 np.float32(89.85931),
 np.float32(58.02561),
 np.float32(84.91374),
 np.float32(67.78187),
 np.float32(59.316486),
 np.float32(76.98073),
 np.float32(70.74645),
 np.float32(83.65092),
 np.float32(63.178825),
 np.float32(69.10569),
 np.float32(86.34047),
 np.float32(74.917946),
 np.float32(56.933395),
 np.float32(72.46589),
 np.float32(69.036545),
 np.float32(83.328354),
 np.float32(63.143566),
 np.float32(69.8716),
 np.float32(79.53